In [14]:
import pandas as pd

import sys, os, requests, json, datetime
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

- pull sms campaigns data from table export (includes list length)
- pull transactions for the relevant forms (status accepted, source sms)
- compile data, calculate conversion rates

### sms campaign data

In [15]:
def fetch_table(table):
    url = 'https://secure.qgiv.com/admin/qgivadmin/statistics/export_tables.php'
    payload = {'key': 'DSQR59VwyFhw21PKDF4K', 'table': table}

    rsp = requests.post(url, data=payload)
    json_data = json.loads(rsp.content)

    return json_data

In [16]:
data = fetch_table("smscampaign")
len(data), len(data[0])

(1, 1531)

In [17]:
smscampaigns = pd.DataFrame(data[0])
smscampaigns['created'] = pd.to_datetime(smscampaigns['created'])
smscampaigns['listlength'] = smscampaigns['listlength'].astype(int)
smscampaigns['org'] = smscampaigns['creatingEntity'].astype(int)
smscampaigns.drop('creatingEntity', axis=1, inplace=True)
smscampaigns.head(3)

,id,created,creatingEntityType,listtype,status,listlength,org
0,1,2019-03-18 13:50:14,3,1,1,0,442662
1,2,2019-03-18 13:53:23,3,1,1,2,442662
2,3,2019-03-18 13:54:59,3,1,1,6,443381


In [18]:
print("{:,} unique orgs".format(len(smscampaigns['org'].unique())))
print("{:,} unique orgs w/ list length > 0".format(len(smscampaigns[smscampaigns['listlength']>0]['org'].unique())))

print("{:,} sms campaigns".format(len(smscampaigns)))
smscampaigns = smscampaigns[smscampaigns['listlength']>0]
print("{:,} sms campaigns w/ list length > 0".format(len(smscampaigns)))

310 unique orgs
262 unique orgs w/ list length > 0
1,531 sms campaigns
1,241 sms campaigns w/ list length > 0


### transactions

In [19]:
q = "select id, form, org, amount, date from transactions where status='A' and source='sms'"
trans = redshift_query_read(q)

In [20]:
trans['org'] = trans['org'].astype(int)
trans['date'] = pd.to_datetime(trans['date'])

In [21]:
print("{:,} rows".format(len(trans)))
trans = trans[trans['org'].isin(smscampaigns['org'].tolist())]
print("{:,} filtered rows to orgs with lists > 0 length".format(len(trans)))

95,579 rows
18,564 filtered rows to orgs with lists > 0 length


### merging data

In [22]:
mrgd = []
for org in smscampaigns['org'].unique():
    org_trans = trans[trans['org']==org]
    org_camps = smscampaigns[smscampaigns['org']==org].copy()
    org_camps.sort_values('created', ascending=True, inplace=True)
    org_camps['endDate'] = org_camps['created'].shift()
    
    for _, campaign in org_camps.iterrows():
        if campaign['endDate'] is not None:
            these_trans = org_trans[(org_trans['date']>=campaign['created'])&(org_trans['date']<campaign['endDate'])]
        else:
            these_trans = org_trans[org_trans['date']>=campaign['created']]
        
        mrgd.append({
            'org': campaign['org'],
            'created': campaign['created'],
            'listlength': campaign['listlength'],
            'total': these_trans['amount'].sum(),
            'count': len(these_trans),
            'first': these_trans['date'].min(),
            'last': these_trans['date'].max()
        })

In [23]:
len(mrgd)

1241

In [24]:
df_mrgd = pd.DataFrame(mrgd)
df_mrgd.head(15)

,org,created,listlength,total,count,first,last
0,442662,2019-03-18 13:53:23,2,0.0,0,NaT,NaT
1,442662,2019-03-19 06:24:42,5,0.0,0,NaT,NaT
2,442662,2019-05-21 16:46:49,2,0.0,0,NaT,NaT
3,442662,2019-08-19 13:20:14,4,0.0,0,NaT,NaT
4,443381,2019-03-18 13:54:59,6,0.0,0,NaT,NaT
5,443381,2019-05-10 15:42:40,4,0.0,0,NaT,NaT
6,443381,2020-02-03 13:41:07,5,0.0,0,NaT,NaT
7,443381,2020-05-18 14:11:15,5,0.0,0,NaT,NaT
8,443381,2020-05-18 14:16:31,2,0.0,0,NaT,NaT
9,443159,2019-03-18 15:27:34,10,0.0,0,NaT,NaT


In [25]:
df_mrgd['total'].value_counts()

0.0    1241
Name: total, dtype: int64

In [ ]:
mrgd = None
for _, i in trans.iterrows():
    try:
        campaign = smscampaigns[(smscampaigns['creatingEntity']==i['org'])&(smscampaigns['created']<i['date'])].sort_values('created', ascending=False).iloc[0]
        i['campaign_created'] = campaign['created']
        i['campaign_listlength'] = campaign['listlength']
    except:
        pass
    if mrgd is None:
        mrgd = i.to_frame()
    else:
        mrgd = mrgd.append(i.to_frame())

In [ ]:
mrgd.head()